In [1]:
#import하기
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#데이터 준비하기 
train_data = pd.read_csv("E:\데이콘\\전복나이 예측\\데이터\\train.csv")
print(train_data.columns)
train_max_input = train_data[['Gender','Lenght','Diameter','Height','Whole Weight','Shucked Weight','Viscra Weight','Shell Weight']]
train_max_target = train_data['Target'].to_numpy()

ques_data = pd.read_csv("E:\데이콘\\전복나이 예측\\데이터\\test.csv")
ques_input = ques_data[['Gender','Lenght','Diameter','Height','Whole Weight','Shucked Weight','Viscra Weight','Shell Weight']]


Index(['id', 'Gender', 'Lenght', 'Diameter', 'Height', 'Whole Weight',
       'Shucked Weight', 'Viscra Weight', 'Shell Weight', 'Target'],
      dtype='object')


In [3]:
#성별 원핫 인코딩
train_max_input = pd.get_dummies(train_max_input)
ques_input =pd.get_dummies(ques_input)

In [4]:
#훈련세트와 테스트 세트 나누기
train_input, test_input, train_target, test_target = train_test_split(train_max_input,train_max_target, test_size=0.2, random_state=42)
#다루기 쉽게 데이터프레임 ㅡ> 넘파이 배열
train_input = train_input.to_numpy()
test_input = test_input.to_numpy()

In [5]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
params = {'learning_rate':[0.1,0.2,0.3,0.05,0.01,0.5],'max_iter':[100,200,300,400]}
gs = GridSearchCV(HistGradientBoostingRegressor(random_state =42),params,n_jobs=-1)
gs.fit(train_input,train_target)
dt = gs.best_estimator_
print(gs.best_params_)

C:\Users\dlehd\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


{'learning_rate': 0.01, 'max_iter': 300}


In [6]:
print(gs.cv_results_['mean_test_score'])

[0.42343013 0.37694403 0.34961012 0.33190152 0.3727042  0.3233329
 0.30217687 0.29090897 0.34625955 0.30322861 0.29090774 0.28486711
 0.46597569 0.42834693 0.40245175 0.38304081 0.42649684 0.48222316
 0.48795926 0.48040427 0.28786929 0.26480825 0.25750864 0.25466608]


In [7]:
print(dt.score(train_input,train_target))
print(dt.score(test_input,test_target))

0.7274884400431437
0.5588244595525652


In [8]:
ans=dt.predict(ques_input)
print(ans)

[ 8.68746527 13.37877024  5.80999798 ...  9.74923415  9.67185296
 11.59215695]


C:\Users\dlehd\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but HistGradientBoostingRegressor was fitted without feature names
  warnings.warn(


In [9]:
##파일에 저장
#f= open("sklearn_HGB_submission.csv","w")
#id = range(1,2925)
#for i in id:
#    f.write(str(i) + ',' + str(ans[i-1]) + '\n')
#f.close()

In [10]:
#예상 평가 점수 만들어보기
def NMAE(true, pred):
    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    return score
pred = dt.predict(test_input)
print(pred[:10])
print(test_target[:10])

print(NMAE(test_target,np.around(pred)))

[10.98754627 12.13225479 10.03639702  8.83796795 12.51326575 11.09321849
  9.40868013  9.57890097  7.43656984 13.3697146 ]
[ 8 13 13  9 13 14 10  9  6 13]
0.15042804728903386
